<a href="https://colab.research.google.com/github/AlgorithmBurv/PML/blob/main/Timeseries_ImamMulyana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library

In [ ]:

import numpy as np
import pandas as pd
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import tensorflow as tf


# Load data

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Membaca data dari file CSV
df = pd.read_csv('/content/Train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23504 entries, 0 to 23503
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  23504 non-null  int64  
 1   date        23504 non-null  object 
 2   co          23504 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 551.0+ KB


In [ ]:
# Koversi tipe date ke date time
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# Cek null
df.isnull().sum()

Unnamed: 0    0
date          0
co            0
dtype: int64

In [ ]:
df.head()

,Unnamed: 0,date,co
0,0,2020-11-30,520.71
1,1,2020-01-12,1682.28
2,2,2020-02-12,1815.80
3,3,2020-03-12,2296.45
4,4,2020-04-12,2189.64


# Normalisasi

In [ ]:
dataset = df["co"]
dataset = pd.DataFrame(dataset)
data = dataset.values

data.shape

(23504, 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (0, 1))
scaled_data = scaler.fit_transform(np.array(data))

# Memisahkan fitur dan target

In [ ]:
dates = df['date'].values
co = scaled_data

In [ ]:

# Skala data (mae < 10% data)
skala_data = (co.max() - co.min()) * 10/100
skala_data

0.09999999999999998

# Memisahkan dataset menjadi training set dan validation set (80% training, 20% validation)

In [ ]:
from sklearn.model_selection import train_test_split
dates_train, dates_test, co_train, co_test = train_test_split(dates, co, test_size=0.2, shuffle=False)

In [ ]:

# Membuat dataset windowed untuk training set
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
# Membuat dataset windowed untuk training set
train_set = windowed_dataset(co, window_size=128, batch_size=500, shuffle_buffer=1000)

# Fungsi callback

In [ ]:
from tensorflow.keras.callbacks import Callback

class AccuracyCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('mae') < 0.09:
            print("\nMAE sudah memenuhi capaian yang dituju.")
            self.model.stop_training = True

accuracy_callback = AccuracyCallback()

# Membuat model

In [ ]:
from tensorflow.keras.layers import LSTM, Bidirectional

model = tf.keras.models.Sequential([
    LSTM(128, return_sequences=True, input_shape=(128, 1)),
    Bidirectional(LSTM(128, kernel_regularizer=tf.keras.regularizers.L2(0.01))),
    tf.keras.layers.Dense(64, activation="sigmoid"),
    tf.keras.layers.Dense(16, activation="sigmoid"),
    tf.keras.layers.Dense(1),
])


In [ ]:
# Kompilasi model
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

In [ ]:
# Pelatihan model
history = model.fit(
    train_set,
    epochs=50,
    callbacks=[accuracy_callback]
    )

Epoch 1/50
47/47 [==============================] - 12s 119ms/step - loss: 4.0586 - mae: 0.1281
Epoch 2/50
47/47 [==============================] - ETA: 0s - loss: 3.9747 - mae: 0.0400
MAE sudah memenuhi capaian yang dituju.
47/47 [==============================] - 8s 175ms/step - loss: 3.9747 - mae: 0.0400
